In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.shape, test.shape

((1460, 81), (1459, 80))

In [3]:
train.describe()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

C:\Users\MO\Anaconda2\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [4]:
train["SalePrice"] = np.log1p(train["SalePrice"])
y_train = train["SalePrice"]

In [5]:
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

In [6]:
from scipy.stats import skew
from scipy.stats.stats import pearsonr
#log transform skewed numeric features:
numeric_feats = train.dtypes[train.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.70]
skewed_feats = skewed_feats.index

In [7]:
'SalePrice' in skewed_feats

False

In [8]:
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])
#Convert categorical variable into dummy/indicator variables
all_data = pd.get_dummies(all_data)
all_data = all_data.fillna(all_data.mean())
all_data.shape

(2919, 288)

In [9]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
X_train.shape, X_test.shape

((1460, 288), (1459, 288))

In [10]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.4, random_state=0)
#X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [17]:
reg = linear_model.RidgeCV(alphas = [0.01, 0.1, 1.0, 10, 16.])
reg.fit(X_train, y_train)
reg.alpha_   

10.0

In [20]:
reg = linear_model.RidgeCV(alphas = [2.0, 5., 10, 16., 50.])
reg.fit(X_train, y_train)
reg.alpha_ 

10.0

In [21]:
reg = linear_model.RidgeCV(alphas = [6., 8., 10., 12., 14., 17.], fit_intercept=True, \
                           normalize=False, scoring=None, cv=10, gcv_mode=None, store_cv_values=False)
reg.fit(X_train, y_train)

RidgeCV(alphas=[6.0, 8.0, 10.0, 12.0, 14.0, 17.0], cv=10, fit_intercept=True,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)

In [22]:
print(reg.alpha_)
y_pred = reg.predict(X_test)
y_pred = np.expm1(y_pred)

10.0


In [23]:
output = pd.DataFrame(test['Id'])

In [24]:
output['SalePrice'] = y_pred

In [25]:
output

,Id,SalePrice
0,1461,119029.139495
1,1462,153034.778689
2,1463,181011.214251
3,1464,197891.150498
4,1465,200308.085076
5,1466,166881.942478
6,1467,178263.220973
7,1468,160936.137123
8,1469,189803.447287
9,1470,121717.178308


In [27]:
output.to_csv('RidgeRegression_2.csv', index=False)